# splash
- javascrip渲染服务，一个带有HTTP API的轻量级浏览器，可以实现动态渲染页面的抓取
- 功能：
    - 异步方式处理多个网页渲染过程
    - 获取渲染后的页面的源代码或截图
    - 通过关闭图片渲染或使用Adblock规则来加快页面渲染速度
    - 可执行特定的Javascript脚本
    - 可通过Lua脚本来控制页面渲染过程
    - 获取渲染的详细过程并通过HAR格式呈现

## 运行

In [ ]:
# 守护形态，关闭docker，仍然会运行
docker run -d -p 8050:8050 -p 5023:5023 scrapinghub/splash
        
docker run -p 8050:8050 -p 5023:5023 scrapinghub/splash

## splash的脚本

### 对象属性

#### args
- 获取加载时配置的参数，比如URL

In [ ]:
function main(splash,args)
    local url = args.url
end

function main(splash)
    local url = splash.args.url
end

#### js_enabled
- javaScript执行开关，可以将其配置为true或false来控制是否执行Javascript代码，默认为true

In [ ]:
function main(splash,args)
    splash:go("http://www.baidu.com")
    splash.js_enabled = false # 禁止javascript执行
    local title = splash:evaljs("document.title") # 执行js语句
    return {title=title}
end

#### resource_timeout
- 此属性可以设置加载的超时时间，单位是秒，如果设置为0或nil（类似PYTHON中的NONE），代表不检测超时

In [ ]:
function main(splash)
    splash.resource_time = 0.1 # 超过这个时间就会报错
    assert(splash:go('https://www.taobao.com'))
    return splash:png()
end

#### images_enabled
- 设置图片是否加载，默认情况加载，禁用该属性后，可节省网络流量并提高网页加载速度

In [ ]:
function main(splash,args)
    splash.images_enabled = false
    assert(splash:go('http://www.jd.com'))
    return {png=splash:png()}
end

#### plugins_enabled
- 属性可以控制浏览器插件是否开启

In [ ]:
function main(splash)
    splash.plugins_enabled = true/false

#### scroll_positon
- 控制页面上下或左右滚动

In [ ]:
function main(splash,args)
    assert(splash:go('https://www.taobao.com'))
    splash.scroll_position = {y=400} #向下滚动400
    return {png=splash:png()}
end

function main(splash,args)
    splash.scroll_position = {x=100,y=200}

### 对象的方法

#### go()
- 请求某个链接，而且它可以模拟GET和POST请求，同事支持传入请求头、表单等数据
- 参数：
    - url：请求的URL
    - baseurl：可选参数，默认为空，表示资源加载相对路径
    - headers：可选参数，默认为空，表示请求头
    - http_method：可选参数，默认为GET，同时支持POST
    - body：可选参数，默认为空，发POST请求时的表单数据，使用的Content-type为application/json
    - formdata：可选参数，默认为空，POST的时候的表单数据，使用的Content-type为application/x-www-form-urlencoded
- 返回值：
    - ok：如果ok为空，代表网页加载出现了错误
    - reason：如果ok为空时，包含了错误的原因

In [ ]:
function main(splash,args)
    local ok,reason = splash:go{"http:httpbin.org/post",http_method="POST",body="name=Germey"}
    if ok then
        return splash:html()
    end
end

#### wait()
- 控制页面的等待时间
- 参数
    - time：等待的秒数
    - cancel_on_redirect：可选参数，默认为false，表示如果发生了重定向就停止等待，并返回重定向结果
    - cancel_on_error：可选参数，默认为false,表示如果发生了加载错误，就停止等待

In [ ]:
function main(splash)
    splash:go("http://www.taobao.com")
    splash:wait(2)
    return {html=splash:html()}
end

#### jsfunc()
- 可以直接调用Javascript定义的方法，但是所调用的方法需要用双中括号包围

In [ ]:
function main(splash,args)
    local get_div_count = splash:jsfunc({
        function(){
            var body = document.body;
            var divs = body.getElementsByTagName('div');
            return divs.length;
        }
    })
    
    splash:go("http://www.baidu.com")
    return ("There are %s DIVs"):format(get_div_count())
end

#### evaljs()
- 执行javascript代码并返回最后一条JavaScript语句的返回结果

In [ ]:
local title = splash:evaljs("document.title")

#### runjs
- 执行js代码，与evaljs功能类似，偏向于执行某些动作或声明某些方法

In [ ]:
function main(splash,args)
    splash:go("http://www.baidu.com")
    splash:runjs("foo=function(){return 'bar'}")
    local result = splash:evaljs("foo()")
    return result
end

#### autoload()
- 可以设置每个页面访问时自动加载的对象，只负责加载js代码或库，不执行任何操作，如果要执行操作，可以调用evaljs()
- 参数
    - source_or_url：js代码或者js库链接
    - source：js代码
    - url：js库链接

In [ ]:
function main(splash,args)
    splash:autoload([[
        function get_document_title(){
            return document.title
        }
    ]])
    splash:go("https://www.baidu.com")
    return splash:evaljs("get_document_title()")
end

#### call_later()
- 可以通过设置定时任务和延迟时间来实现任务延时执行，并且可以在执行前通过cancel()方法重新执行定时任务

In [ ]:
function main(splash,args)
    local snapshots = {}
    local timer = splash:call_later(function()
       snapshots["a"] = splash:png() # 在0.2秒的时候，截图
       splash:wait(1.0)  # 等待1秒
       snapshots["b"] = splash:png() # 在1.2秒时，截图
    end,0.2)
    splash:go("https://www.taobao.com")
    splash:wait(3.0)
    return snapshots
end

#### http_get()
- 模拟发送HTTP的GET请求
- 参数
    - url：请求URL
    - headers：可选参数，默认为空，请求头
    - follow_redirects：可选参数，表示是否启动自动重定向，默认为true

In [ ]:
function main(splash,args)
    local treat = require("treat")
    local response = splash:http_get("http://httpbin.org/get")
        return {
            html = treat.as_string(response.body),
            url = response.url,
            status = response.status
        }
end

#### http_post()
- 用来模拟发送POST请求，不过多了一个参数body
- 参数
    - url：请求url
    - headers：可选参数，默认为空，请求头
    - follow_redirects：可选参数，表示是否启动自动重定向，默认为true
    - body：可选参数，即表单数据，默认为空

In [ ]:
function main(splash,args)
    local treat = require("treat")
    local json = require("json")
    local response = splash:http_post{
    		"http://httpbin.org/post",
        body = json.encode({name="Germey"}),
        headers = {["content-type"]="application/json"}
  	}
    return {
        html = treat.as_string(response.body),
        url = response.url,
        status = response.status
    }
end

#### set_content()
- 设置页面的内容

In [ ]:
function main(splash)
    assert(splash:set_content("<html><body>1</body></html>"))
    return splash:png()
end

#### html
- 获取网页的源代码

In [ ]:
function main(splash,args)
    splash:go("http://httpbin.org/get")
    return splash:html()
end

#### png
- 获取png格式的网页截图

In [ ]:
function main(splash,args)
    splash:go("http:///www.taobao.com")
    return splash:png() # 必须作为返回值，否则无法展示
end

#### jpeg()
- 获取jpeg格式的网页截图

In [ ]:
function main(splash,args)
    splash:go("http:///www.taobao.com")
    splash:wait(10)
  	return {
    	jpeg1=splash:jpeg(),
    	png=splash:png()
  }
end

#### har
- 获取页面加载过程描述

In [ ]:
function main(splash,args)
    splash:go("http://www.baidu.com")
    return splash:har()
end

#### url
- 获取当前正在访问的url

In [ ]:
function main(splash,args)
    splash:go("https://www.baidu.com")
    return splash:url()
end

#### get_cookies()
- 获取当前页面的cookies

In [ ]:
function main(splash,args)
    splash:go("https://www.baidu.com")
    return splash:get_cookies()
end

#### add_cookie()
- 为当前页面添加cookie

In [ ]:
function main(splash)
    splash:add_cookie{"sessionid","123","/",domain="http://example.com"}
    splash:go("http://example.com/")
    return splash:html()
end

#### clear_cookies()
- 清除所有的cookies

In [ ]:
function main(splash)
    splash:go("https://www.baidu.com/")
    splash:clear_cookies()
    return splash:get_cookies()
end

#### get_viewpor_size()
- 可以获取当前浏览器页面的大小，即宽高

In [ ]:
function main(splash)
    splash:go("https://www.baidu.com/")
    return splash:get_viewport_size()
end

#### set_viewport_size()
- 设置当前浏览器页面的大小，即宽高

In [ ]:
function main(splash)
    splash:set_viewport_size(400,700)
    assert(splash:go("https://cuiqingcai.com"))
    return splash:png()
end

#### set_viewport_full
- 设置浏览器全屏显示

In [ ]:
function main(splash)
    splash:set_viewport_full()
    assert(splash:go("http://cuiqingcai.com"))
    return splash:png()
end

#### set_user_agent()
- 设置浏览器的User-Agent

In [ ]:
function main(splash)
    splash:set_user_agent('Splash')
    splash:go("http://httpbin.org/get")
    return splash:html()
end

#### set_custom_headers()
- 设置请求头

In [ ]:
function main(splash)
    splash:set_custom_headers({
        ["User-Agent"] = "Splash",
        ["Site"] = "Splash",
    })
    splash:go("http://httpbin.org/get")
    return splash:html()
end

#### select()
- 选中符合条件的第一个节点，如果有多个节点符合条件，则只会返回一个，其参数为css选择器

In [ ]:
function main(splash)
    splash:go("https://www.baidu.com/")
    input = splash:select("#kw")
    input:send_text('Splash')
    splash:wait(3)
    return splash:png()
end

#### select_all()
- 选中所有符合条件的节点，参数为css选择器

In [ ]:
function main(splash)
    local treat = require('treat')
    assert(splash:go("http://quotes.toscrape.com/"))
    local texts = splash:select_all('.quote .text')
    local results = {}
    for index,text in ipairs(texts) do
        results[index] = text.node.innerHTML
    end
    return treat.as_array(results)
end

#### mouse_click()
- 模拟用户点击操作，传入的参数为坐标值x和y

In [ ]:
function main(splash)
    splash:go("http://www.baidu.com")
    input = splash:select("#kw")
    input:send_text('Splash')
    submit = splash:select('#su')
    submit:mouse_click()
    splash:wait(3)
    return splash:png()
end

## splash API

#### render.html
- javascript渲染的页面的html代码

In [ ]:
import requests
url = 'http://localhost:8050/render.html?url=https://www.baidu.com'
# 增加等待时间
url = 'http://localhost:8050/render.html?url=https://www.baidu.com&wait=5'
response = requests.get(url)
print(response.text)

#### render.png
- 获取网页截图，返回PNG格式的图片二进制数据

In [ ]:
import requests
url = 'http://localhost:8050/render.png?url=https://www.jd.com&wait=5&width=1000&height=700'
response = requests.get(url)
with open('taobao.png','wb') as f:
    f.write(response.content)

#### render.jpeg

#### render.har
- 用于获取页面加载的HAR数据，一个json格式的数据

#### render.json
- 包含了前面接口的所有功能，返回结果是JSON格式

#### execute
- 可以执行Lua脚本

In [ ]:
import requests
from urllib.parse import quote
lua = '''
function main(splash,args)
    local treat = require("treat")
    local response = splash:http_get("http://httpbin.org/get")
    return {
        html = treat.as_string(response.body),
        url = response.url,
        status = response.status
    }
end
'''
url = 'http://localhost:8050/execute?lua_source='+quote(lua)
response = requests.get(url)
print(response.text)

## splash负载均衡
- 配置Splash服务
- 配置负载均衡

In [ ]:
# Nginx的配置文件nginx.conf
http{
    upstream splash{
        least_conn; # 最小链接负载均衡
        server 41.159.27.233:8050 weight=4; # weight配置权重，权重越高，分配到处理的请求越多
        server 41.159.27.221:8050 weight=2;
    }
    server{
        listen 8050;
        location / {
            proxy_pass http://splash;
            auth_basic "Restricted";
            auth_basic_user_file /etc/nginx/conf.d/.htpasswd
        }
    }
}